In [1]:
#Import modules
import numpy as np
import holidays
import pandas as pd
import datetime
import sklearn
import scipy
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import math
from sklearn.model_selection import cross_val_score
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [2]:
df1 = pd.read_csv('Accident_Information.csv')
df2 = pd.read_csv('Vehicle_Information.csv')

In [3]:
print(df1.columns)
print(df2.columns)

Index(['Accident_Index', '1st_Road_Class', '1st_Road_Number', '2nd_Road_Class',
       '2nd_Road_Number', 'Accident_Severity', 'Carriageway_Hazards', 'Date',
       'Day_of_Week', 'Did_Police_Officer_Attend_Scene_of_Accident',
       'Junction_Control', 'Junction_Detail', 'Latitude', 'Light_Conditions',
       'Local_Authority_(District)', 'Local_Authority_(Highway)',
       'Location_Easting_OSGR', 'Location_Northing_OSGR', 'Longitude',
       'LSOA_of_Accident_Location', 'Number_of_Casualties',
       'Number_of_Vehicles', 'Pedestrian_Crossing-Human_Control',
       'Pedestrian_Crossing-Physical_Facilities', 'Police_Force',
       'Road_Surface_Conditions', 'Road_Type', 'Special_Conditions_at_Site',
       'Speed_limit', 'Time', 'Urban_or_Rural_Area', 'Weather_Conditions',
       'Year', 'InScotland'],
      dtype='object')
Index(['Accident_Index', 'Age_Band_of_Driver', 'Age_of_Vehicle',
       'Driver_Home_Area_Type', 'Driver_IMD_Decile', 'Engine_Capacity_.CC.',
       'Hit_Object_i

In [5]:
print("df1 shape:", df1.shape)
print("df2 shape:", df2.shape)

df1 shape: (2047256, 34)
df2 shape: (2177205, 24)


In [6]:
df2.Age_of_Vehicle.max()

111.0

In [7]:
df3 = df1.merge(df2, on='Accident_Index')

In [8]:
df3.head()

,Accident_Index,1st_Road_Class,1st_Road_Number,2nd_Road_Class,2nd_Road_Number,Accident_Severity,Carriageway_Hazards,Date,Day_of_Week,Did_Police_Officer_Attend_Scene_of_Accident,...,Skidding_and_Overturning,Towing_and_Articulation,Vehicle_Leaving_Carriageway,Vehicle_Location.Restricted_Lane,Vehicle_Manoeuvre,Vehicle_Reference,Vehicle_Type,Was_Vehicle_Left_Hand_Drive,X1st_Point_of_Impact,Year_y
0,200501BS00002,B,450.0,C,0.0,Slight,None,2005-01-05,Wednesday,1.0,...,None,No tow/articulation,Did not leave carriageway,0.0,Slowing or stopping,1,Bus or coach (17 or more pass seats),No,Nearside,2005
1,200501BS00003,C,0.0,NaN,0.0,Slight,None,2005-01-06,Thursday,1.0,...,None,No tow/articulation,Did not leave carriageway,0.0,Going ahead right-hand bend,1,Bus or coach (17 or more pass seats),No,Nearside,2005
2,200501BS00004,A,3220.0,NaN,0.0,Slight,None,2005-01-07,Friday,1.0,...,None,No tow/articulation,Did not leave carriageway,0.0,Going ahead other,1,Car,No,Front,2005
3,200501BS00005,Unclassified,0.0,NaN,0.0,Slight,None,2005-01-10,Monday,1.0,...,Skidded,No tow/articulation,Did not leave carriageway,0.0,Going ahead other,1,Motorcycle 125cc and under,No,Front,2005
4,200501BS00006,Unclassified,0.0,NaN,0.0,Slight,None,2005-01-11,Tuesday,1.0,...,None,No tow/articulation,Did not leave carriageway,0.0,Moving off,1,Car,No,Did not impact,2005


In [9]:
df3.describe()

,1st_Road_Number,2nd_Road_Number,Did_Police_Officer_Attend_Scene_of_Accident,Latitude,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Number_of_Casualties,Number_of_Vehicles,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Speed_limit,Year_x,Age_of_Vehicle,Driver_IMD_Decile,Engine_Capacity_.CC.,Vehicle_Location.Restricted_Lane,Vehicle_Reference,Year_y
count,2.058407e+06,2.039481e+06,2.058294e+06,2.058284e+06,2.058284e+06,2.058284e+06,2.058283e+06,2.058408e+06,2.058408e+06,2.057754e+06,2.057038e+06,2.058343e+06,2.058408e+06,1.720446e+06,1.369118e+06,1.808091e+06,2.057284e+06,2.058408e+06,2.058408e+06
mean,9.820954e+02,3.809252e+02,1.174720e+00,5.254775e+01,4.422955e+05,2.955414e+05,-1.397291e+00,1.449605e+00,2.124032e+00,8.732822e-03,7.270488e-01,3.975699e+01,2.011334e+03,7.143248e+00,5.385381e+00,2.028291e+03,1.088396e-01,1.553280e+00,2.011334e+03
std,1.789819e+03,1.298916e+03,3.871537e-01,1.437453e+00,9.488500e+04,1.596222e+05,1.393690e+00,1.033484e+00,9.641580e-01,1.238491e-01,1.814948e+00,1.460916e+01,3.391307e+00,4.728746e+00,2.820296e+00,1.914150e+03,8.873613e-01,7.767501e-01,3.391307e+00
min,0.000000e+00,0.000000e+00,1.000000e+00,4.991294e+01,6.495000e+04,1.029000e+04,-7.516225e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.005000e+03,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,2.005000e+03
25%,0.000000e+00,0.000000e+00,1.000000e+00,5.148191e+01,3.805615e+05,1.772900e+05,-2.291969e+00,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,3.000000e+01,2.009000e+03,3.000000e+00,3.000000e+00,1.299000e+03,0.000000e+00,1.000000e+00,2.009000e+03
50%,1.260000e+02,0.000000e+00,1.000000e+00,5.221756e+01,4.437000e+05,2.593670e+05,-1.352466e+00,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,3.000000e+01,2.012000e+03,7.000000e+00,5.000000e+00,1.598000e+03,0.000000e+00,1.000000e+00,2.012000e+03
75%,6.900000e+02,0.000000e+00,1.000000e+00,5.345394e+01,5.238690e+05,3.954200e+05,-2.106275e-01,2.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,5.000000e+01,2.014000e+03,1.000000e+01,8.000000e+00,1.997000e+03,0.000000e+00,2.000000e+00,2.014000e+03
max,9.999000e+03,9.999000e+03,3.000000e+00,6.075754e+01,6.553700e+05,1.208800e+06,1.759382e+00,9.300000e+01,6.700000e+01,2.000000e+00,8.000000e+00,7.000000e+01,2.016000e+03,1.110000e+02,1.000000e+01,9.600000e+04,9.000000e+00,9.100000e+01,2.016000e+03


In [10]:
print("df3 shape:",df3.shape)

df3 shape: (2058408, 57)


In [11]:
df3.isnull().sum().sort_values(ascending=False)/2058408*100

2nd_Road_Class                                 40.222201
Driver_IMD_Decile                              33.486559
Age_of_Vehicle                                 16.418611
model                                          14.543132
Engine_Capacity_.CC.                           12.160709
Propulsion_Code                                11.348479
LSOA_of_Accident_Location                       6.762848
make                                            5.384987
2nd_Road_Number                                 0.919497
Pedestrian_Crossing-Physical_Facilities         0.066556
Vehicle_Location.Restricted_Lane                0.054605
Pedestrian_Crossing-Human_Control               0.031772
Time                                            0.007093
Longitude                                       0.006073
Location_Easting_OSGR                           0.006024
Location_Northing_OSGR                          0.006024
Latitude                                        0.006024
Did_Police_Officer_Attend_Scene

In [12]:
df3 = df3.drop(['2nd_Road_Class', 'Driver_IMD_Decile','Year_y'], axis=1)

df3 = df3.rename(columns={'Year_x':'Year'})

In [13]:
#standardize all columns to lowercase, and remove some characters 
#for ease of use in querying
df3.columns = map(str.lower, df3.columns)
df3.columns = df3.columns.str.replace('.','')
df3.columns = df3.columns.str.replace('(','')
df3.columns = df3.columns.str.replace(')','')
#verify
df3.columns

Index(['accident_index', '1st_road_class', '1st_road_number',
       '2nd_road_number', 'accident_severity', 'carriageway_hazards', 'date',
       'day_of_week', 'did_police_officer_attend_scene_of_accident',
       'junction_control', 'junction_detail', 'latitude', 'light_conditions',
       'local_authority_district', 'local_authority_highway',
       'location_easting_osgr', 'location_northing_osgr', 'longitude',
       'lsoa_of_accident_location', 'number_of_casualties',
       'number_of_vehicles', 'pedestrian_crossing-human_control',
       'pedestrian_crossing-physical_facilities', 'police_force',
       'road_surface_conditions', 'road_type', 'special_conditions_at_site',
       'speed_limit', 'time', 'urban_or_rural_area', 'weather_conditions',
       'year', 'inscotland', 'age_band_of_driver', 'age_of_vehicle',
       'driver_home_area_type', 'engine_capacity_cc',
       'hit_object_in_carriageway', 'hit_object_off_carriageway',
       'journey_purpose_of_driver', 'junction_l

In [14]:
year = range(2005,2018)
uk_holidays = holidays.UnitedKingdom(years=year)

In [15]:
uk_holidays.years

{2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017}

In [16]:
ukh  = pd.DataFrame(list(uk_holidays.items()))

In [17]:
ukh.dtypes

0    object
1    object
dtype: object

In [18]:
ukh = ukh.rename(columns={0:"date", 1:"holiday"})

In [19]:
df = pd.concat([df3,ukh])

In [20]:
df.head()

,1st_road_class,1st_road_number,2nd_road_number,accident_index,accident_severity,age_band_of_driver,age_of_vehicle,carriageway_hazards,date,day_of_week,...,urban_or_rural_area,vehicle_leaving_carriageway,vehicle_locationrestricted_lane,vehicle_manoeuvre,vehicle_reference,vehicle_type,was_vehicle_left_hand_drive,weather_conditions,x1st_point_of_impact,year
0,B,450.0,0.0,200501BS00002,Slight,36 - 45,3.0,None,2005-01-05,Wednesday,...,Urban,Did not leave carriageway,0.0,Slowing or stopping,1.0,Bus or coach (17 or more pass seats),No,Fine no high winds,Nearside,2005.0
1,C,0.0,0.0,200501BS00003,Slight,26 - 35,5.0,None,2005-01-06,Thursday,...,Urban,Did not leave carriageway,0.0,Going ahead right-hand bend,1.0,Bus or coach (17 or more pass seats),No,Fine no high winds,Nearside,2005.0
2,A,3220.0,0.0,200501BS00004,Slight,46 - 55,4.0,None,2005-01-07,Friday,...,Urban,Did not leave carriageway,0.0,Going ahead other,1.0,Car,No,Fine no high winds,Front,2005.0
3,Unclassified,0.0,0.0,200501BS00005,Slight,46 - 55,10.0,None,2005-01-10,Monday,...,Urban,Did not leave carriageway,0.0,Going ahead other,1.0,Motorcycle 125cc and under,No,Fine no high winds,Front,2005.0
4,Unclassified,0.0,0.0,200501BS00006,Slight,46 - 55,1.0,None,2005-01-11,Tuesday,...,Urban,Did not leave carriageway,0.0,Moving off,1.0,Car,No,Raining no high winds,Did not impact,2005.0


In [21]:
#Check Missing Data
df.isnull().sum().sort_values(ascending=False)

holiday                                        2058408
age_of_vehicle                                  338152
model                                           299547
engine_capacity_cc                              250507
propulsion_code                                 233788
lsoa_of_accident_location                       139397
make                                            111035
2nd_road_number                                  19117
pedestrian_crossing-physical_facilities           1560
vehicle_locationrestricted_lane                   1314
pedestrian_crossing-human_control                  844
time                                               336
longitude                                          315
latitude                                           314
location_northing_osgr                             314
location_easting_osgr                              314
did_police_officer_attend_scene_of_accident        304
speed_limit                                        255
inscotland

In [22]:
#filled null holiday spots with "Non-Holiday"
df['holiday'].fillna("Non-Holiday", inplace=True)

#dropped null 'accident_index' rows that came about after adding holidays to the dataframe
df = df.dropna(subset=['accident_index'])

In [23]:
print('df shape', df.shape)

df shape (2058408, 55)


In [24]:
#Check Missing Data
df.isnull().sum().sort_values(ascending=False)/2058508*100

age_of_vehicle                                 16.417813
model                                          14.542426
engine_capacity_cc                             12.160118
propulsion_code                                11.347928
lsoa_of_accident_location                       6.762519
make                                            5.384725
2nd_road_number                                 0.919452
pedestrian_crossing-physical_facilities         0.066553
vehicle_locationrestricted_lane                 0.054603
pedestrian_crossing-human_control               0.031771
time                                            0.007093
longitude                                       0.006072
latitude                                        0.006024
location_easting_osgr                           0.006024
location_northing_osgr                          0.006024
did_police_officer_attend_scene_of_accident     0.005538
speed_limit                                     0.003158
inscotland                     

In [25]:
df.describe()

,1st_road_number,2nd_road_number,age_of_vehicle,did_police_officer_attend_scene_of_accident,engine_capacity_cc,latitude,location_easting_osgr,location_northing_osgr,longitude,number_of_casualties,number_of_vehicles,pedestrian_crossing-human_control,pedestrian_crossing-physical_facilities,speed_limit,vehicle_locationrestricted_lane,vehicle_reference,year
count,2.058407e+06,2.039481e+06,1.720446e+06,2.058294e+06,1.808091e+06,2.058284e+06,2.058284e+06,2.058284e+06,2.058283e+06,2.058408e+06,2.058408e+06,2.057754e+06,2.057038e+06,2.058343e+06,2.057284e+06,2.058408e+06,2.058408e+06
mean,9.820954e+02,3.809252e+02,7.143248e+00,1.174720e+00,2.028291e+03,5.254775e+01,4.422955e+05,2.955414e+05,-1.397291e+00,1.449605e+00,2.124032e+00,8.732822e-03,7.270488e-01,3.975699e+01,1.088396e-01,1.553280e+00,2.011334e+03
std,1.789819e+03,1.298916e+03,4.728746e+00,3.871537e-01,1.914150e+03,1.437453e+00,9.488500e+04,1.596222e+05,1.393690e+00,1.033484e+00,9.641580e-01,1.238491e-01,1.814948e+00,1.460916e+01,8.873613e-01,7.767501e-01,3.391307e+00
min,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,4.991294e+01,6.495000e+04,1.029000e+04,-7.516225e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.005000e+03
25%,0.000000e+00,0.000000e+00,3.000000e+00,1.000000e+00,1.299000e+03,5.148191e+01,3.805615e+05,1.772900e+05,-2.291969e+00,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,3.000000e+01,0.000000e+00,1.000000e+00,2.009000e+03
50%,1.260000e+02,0.000000e+00,7.000000e+00,1.000000e+00,1.598000e+03,5.221756e+01,4.437000e+05,2.593670e+05,-1.352466e+00,1.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,3.000000e+01,0.000000e+00,1.000000e+00,2.012000e+03
75%,6.900000e+02,0.000000e+00,1.000000e+01,1.000000e+00,1.997000e+03,5.345394e+01,5.238690e+05,3.954200e+05,-2.106275e-01,2.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,5.000000e+01,0.000000e+00,2.000000e+00,2.014000e+03
max,9.999000e+03,9.999000e+03,1.110000e+02,3.000000e+00,9.600000e+04,6.075754e+01,6.553700e+05,1.208800e+06,1.759382e+00,9.300000e+01,6.700000e+01,2.000000e+00,8.000000e+00,7.000000e+01,9.000000e+00,9.100000e+01,2.016000e+03


In [51]:
df.corr()

,1st_road_number,2nd_road_number,age_of_vehicle,did_police_officer_attend_scene_of_accident,engine_capacity_cc,latitude,location_easting_osgr,location_northing_osgr,longitude,number_of_casualties,number_of_vehicles,pedestrian_crossing-human_control,pedestrian_crossing-physical_facilities,speed_limit,vehicle_locationrestricted_lane,vehicle_reference,year
1st_road_number,1.000000,0.185187,0.011088,0.000488,-0.026074,0.045352,-0.091314,0.044401,-0.089017,-0.002647,-0.031816,0.004112,0.041880,-0.089424,-0.003155,-0.020887,-0.008466
2nd_road_number,0.185187,1.000000,-0.005693,0.022242,-0.009651,0.026489,-0.010854,0.025821,-0.009308,-0.007387,-0.017126,0.003649,0.097831,-0.052136,-0.016912,-0.012473,-0.011667
age_of_vehicle,0.011088,-0.005693,1.000000,0.008725,-0.077762,-0.069871,0.020823,-0.069784,0.022433,-0.004048,-0.041111,-0.003985,-0.006598,-0.033292,0.000125,-0.048941,0.158269
did_police_officer_attend_scene_of_accident,0.000488,0.022242,0.008725,1.000000,0.017738,-0.010322,0.022681,-0.010392,0.023099,-0.098979,-0.082738,0.008863,0.012790,-0.140432,0.017932,-0.052175,0.029434
engine_capacity_cc,-0.026074,-0.009651,-0.077762,0.017738,1.000000,0.026077,-0.000357,0.026003,-0.000793,0.013744,-0.004566,0.005127,0.010251,0.078290,0.025870,-0.001583,-0.069400
latitude,0.045352,0.026489,-0.069871,-0.010322,0.026077,1.000000,-0.418207,0.999973,-0.428465,0.027727,-0.022231,0.038625,-0.009157,0.045885,0.009774,-0.009881,-0.033288
location_easting_osgr,-0.091314,-0.010854,0.020823,0.022681,-0.000357,-0.418207,1.000000,-0.416185,0.999367,-0.031665,0.004655,-0.024969,0.056477,-0.055032,-0.002367,0.000451,0.046787
location_northing_osgr,0.044401,0.025821,-0.069784,-0.010392,0.026003,0.999973,-0.416185,1.000000,-0.426542,0.027590,-0.022360,0.038681,-0.009262,0.046086,0.009871,-0.009942,-0.033131
longitude,-0.089017,-0.009308,0.022433,0.023099,-0.000793,-0.428465,0.999367,-0.426542,1.000000,-0.030479,0.005343,-0.026111,0.055114,-0.054101,-0.002985,0.000795,0.046408
number_of_casualties,-0.002647,-0.007387,-0.004048,-0.098979,0.013744,0.027727,-0.031665,0.027590,-0.030479,1.000000,0.382233,-0.006519,-0.027376,0.148506,-0.020343,0.243450,-0.014484


In [34]:
df = df.drop(['location_easting_osgr', 'location_northing_osgr'], axis=1)


,location_easting_osgr,location_northing_osgr,latitude,longitude
0,524170.0,181650.0,51.520075,-0.211708
1,524520.0,182240.0,51.525301,-0.206458
2,526900.0,177530.0,51.482442,-0.173862
3,528060.0,179040.0,51.495752,-0.156618
4,524770.0,181160.0,51.515540,-0.203238
5,524770.0,181160.0,51.515540,-0.203238
6,524220.0,180830.0,51.512695,-0.211277
7,524220.0,180830.0,51.512695,-0.211277
8,525890.0,179710.0,51.502260,-0.187623
9,526240.0,178900.0,51.494902,-0.182872
